In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import skfuzzy
from skfuzzy import control as ctrl
import warnings

In [2]:
warnings.filterwarnings('ignore')

# Carga de datos

En caso de necesitar acceder a datos externos

In [ ]:
datos = pd.read_csv('nombre_archivo.csv' #,sep = ';'#)
datos.head()

In [ ]:
datos = datos[['col1', 'col2', 'col3']] #Seleccionar columnas útiles
datos.rename(columns = {'col1':'var1', 'col2':'var2', 'col3':'var3'}, inplace = True) #Renombrar columnas (opcional)
datos.head()

In [ ]:
datos.info()

## Análisis variable 1

Repetir para el resto de variables

In [ ]:
datos.nombre1.describe() #Identificar mínimo y máximo (universo del discurso)

In [ ]:
#Calcular percentiles y deciles (reemplazar nombre1)
percentil_var1 = []
for i in range(0, 110, 10):
    percentil_var1.append(np.percentile(datos.var1, i))
cuartil_var1 = []
for i in [0, 0.25, 0.5, 0.75, 1]:
    cuartil_var1.append(np.quantile(datos.var1, i))

sb.histplot(data = datos, x = 'var1', kde = True)
for i in range(len(percentil_var1)):
    plt.axvline(percentil_var1[i], color = 'red')
for i in range(len(cuartil_var1)):
    plt.axvline(cuartil_var1[i], color = 'green')
    
plt.show()

In [ ]:
print('Valor del percentil: ', percentil_var1)
print('Valor del cuartil: ', cuartil_var1)

In [ ]:
px.box(data_frame = datos, y = 'var1')

# Modelo de inferencia borrosa

1. Universo del discurso

In [ ]:
#SI SE TOMA DE UN SET DE DATOS:
#Antecedente 1
var1 = ctrl.Antecedent(np.arange(datos.var1.min(), datos.nombre2.max() + 1, 1), 'var1') 
#Antecedente 2
var2 = ctrl.Antecedent(np.arange(datos.var2.min(), datos.var2.max() + 1, 1), 'var2')
#Consecuente
var3 = ctrl.Consequent(np.arange(datos.var3.min(), datos.var3.max() + 1, 1), 'var3')

#SI NO:
#Antecedente 1
var1 = ctrl.Antecedent(np.arange(0,1001, 1),var1")
#Antecedente 2
var2 = ctrl.Antecedent(np.arange(-10,10,0.1), var2")
#Consecuente
var3 = ctrl.Consequent(np.arange(0,11,1), var3")

2. Fuzzificar antecedentes y consecuentes

In [ ]:
#trapmf -> Trapecio
#trimf -> triángulo
#sigmf(500: punto de inflexión, -0.01: ancho. Signo indica si aciende o desciende)  -> Sigmoidea
#gaussmf(media, desviación) -> Gaussiana

#SI SE TOMA DE UN SET DE DATOS:
#var1
var1['categoria1'] = skfuzzy.trapmf(var1.universe, [cuartil_var1[0], cuartil_var1[0], 
                                               cuartil_var1[1], cuartil_var1[2]])
var1['categoria2'] = skfuzzy.trapmf(var1.universe, [percentil_Edad[3], percentil_Edad[5], 
                                                percentil_Edad[6], percentil_Edad[7]])

#SI NO:
#var1
var1['categoria1'] = skfuzzy.trapmf(var1.universe,[400,500,600,600])
var1['categoria2'] = skfuzzy.trimf(var1.universe, [200,350,450])



var1.view()
plt.show

3. Crear base de conocimiento

In [ ]:
regla1 = ctrl.Rule(var1['categoria1'] & var2['baja'], prima['muy alta'])
regla2 = ctrl.Rule(var1['categoria1'] & var2['media baja'], prima['alta'])
regla3 = ctrl.Rule(var1['categoria1'] & var2['media alta'], prima['media'])
regla4 = ctrl.Rule(var1['categoria1'] & var2['alta'], prima['media'])
regla5 = ctrl.Rule(var1['categoria2'] & var2['baja'], prima['media'])
regla6 = ctrl.Rule(var1['categoria2'] & var2['media baja'], prima['baja'])
regla7 = ctrl.Rule(var1['categoria2'] & var2['media alta'], prima['muy baja'])
regla8 = ctrl.Rule(var1['categoria2'] & var2['alta'], prima['muy baja'])
regla9 = ctrl.Rule(var1['categoria3'] & var2['baja'], prima['muy alta'])
regla10 = ctrl.Rule(var1['categoria3'] & var2['media baja'], prima['alta'])
regla11 = ctrl.Rule(var1['categoria3'] & var2['media alta'], prima['media'])
regla12 = ctrl.Rule(var1['categoria3'] & var2['alta'], prima['baja'])

4. Activar motor de inferencia

In [ ]:
calculo = ctrl.ControlSystem([regla1, regla2, regla3, regla4,
                                  regla5, regla6, regla7, regla8,
                                  regla9, regla10, regla11, regla12,])

5. Utilizar el SIB

In [ ]:
calcular = ctrl.ControlSystemSimulation(calculo)
calcular.input['var1'] = 40
calcular.input['var2'] = 120
calcular.compute()
calcular.output['var3']

In [ ]:
var1.view(sim = calcular)
var2.view(sim = calcular)
var3.view(sim = calcular)